In [ ]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


## Instalando pacotes necessários

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install triton
!pip install xformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-d5hhkbfj/unsloth_7fb94d1c66684538968cc78bea8b6acd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-d5hhkbfj/unsloth_7fb94d1c66684538968cc78bea8b6acd
  Resolved https://github.com/unslothai/unsloth.git to commit 3dff3b38687c92cfbe80a62324eadccb4672206e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   

## Importando bibliotecas utilizadas

In [ ]:
import torch
import pandas as pd
import time
from trl import SFTTrainer
from datasets import load_dataset
from datasets import Dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
from transformers import TrainerCallback

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Justificativa

## Escolha do Modelo LLM Utilizando LLaMA 3
O modelo LLaMA 3 foi escolhido por apresentar diversos fatores vantajosos, como código open source, a gratuidade, facilidade de modificação e personalização, além de sua alta performance e eficiência em tarefas de processamento de linguagem natural.

1. **Open Source**
  - O LLaMA 3 é um modelo de código aberto, o que significa que o código-fonte está disponível para a comunidade. Essa característica promove a transparência no desenvolvimento e permite que desenvolvedores e pesquisadores possam: Inspecionar e entender o funcionamento interno do modelo.
  - Contribuir para melhorias e otimizações contínuas.
  - Adaptar o modelo para necessidades específicas, promovendo inovação colaborativa.

2. **Grauidade**

  - Optar por um modelo gratuito como o LLaMA 3 reduz significativamente os custos associados ao desenvolvimento e implementação de soluções baseadas em inteligência artificial.
  - Facilidade de experimentação sem a necessidade de investimentos financeiros elevados.
  - Redução de barreiras para a entrada de novos participantes no campo de machine learning e deep learning.

3. **Facilidade de Modificação e Customização**

  - A natureza open source do LLaMA 3 facilita a personalização e modificação do modelo conforme as necessidades específicas do projeto.
  - Ajustes finos (fine-tuning) para melhorar o desempenho em tarefas específicas.
  - Integração com diferentes frameworks e ferramentas de machine learning.
  - Implementação de novas funcionalidades ou melhorias sem restrições impostas por licenças proprietárias.

4. **Performance e Eficiência**

  - O LLaMA 3 demonstra um desempenho robusto em diversas tarefas de processamento de linguagem natural (NLP), equilibrando eficiência e precisão. As vantagens adicionais são:
  - Escalabilidade, permitindo o treinamento e a implementação em diferentes escalas de dados e complexidade.
  - Otimizações que aproveitam melhor os recursos computacionais disponíveis, especialmente quando utilizados em conjunto com GPUs no Google Colab.

5. **Compatibilidade com Infraestruturas Existentes**

  - O LLaMA 3 é compatível com diversas infraestruturas e ferramentas populares no ecossistema de machine learning, facilitando a sua integração com pipelines de desenvolvimento já estabelecidos.
  - Compatibilidade com TensorFlow e PyTorch, dois dos frameworks mais utilizados na área.
  - Facilidade de deploy em diferentes ambientes, seja em nuvem ou em servidores locais.
  - Interoperabilidade com ferramentas de gerenciamento de modelos e monitoramento de desempenho.

**Referências:**

META AI. LLaMA 3: Open Source Language Model. Disponível em: https://ai.meta.com/llama/. Acesso em: 25 abr. 2024.


## (TESTE) Carregando LLama3 8B Instruct

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = 8192,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

## (TESTE) Definindo tokenizer

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

## (TESTE) Fazendo inferência com o LLama3 8B Instruct

In [ ]:
messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The Fibonacci sequence continues as follows:

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144.<|eot_id|>


## (TESTE) Testando LLama3 8B Instruct com perguntas da Brastel

In [ ]:
messages = [
    {"from": "human", "value": "O que você sabe sobre a empresa japonesa Brastel?"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

O que você sabe sobre a empresa japonesa Brastel?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Infelizmente, não tenho informações atualizadas sobre a empresa japonesa Brastel, pois não há muitas informações disponíveis sobre essa empresa. No entanto, posso tentar fornecer algumas informações gerais.

A Brastel é uma empresa japonesa que oferece serviços de comunicação, incluindo telefonia móvel, serviços de internet e outros serviços de comunicação. Ela parece ser uma empresa menor e especializada em serviços de comunicação para empresas e indivíduos.

Se você está procurando informações específicas sobre a Brastel, recomendo que você verifique os sites oficiais da empresa ou busque informações em portais de negócios japoneses. Além disso, se você tiver acesso a informações mais recentes, por favor,

In [ ]:
messages = [
    {"from": "human", "value": "Como fazer uma remessa com o Brastel Remit?"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Como fazer uma remessa com o Brastel Remit?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Para fazer uma remessa com o Brastel Remit, você precisa seguir os seguintes passos:

1. **Crie uma conta**: Acesse o site do Brastel Remit e crie uma conta. Você precisará fornecer informações como nome, e-mail e senha.
2. **Selecione o tipo de remessa**: Escolha o tipo de remessa que você deseja fazer (por exemplo, dinheiro, cartões, etc.).
3. **Insira os detalhes da remessa**: Forneça os detalhes da remessa, como o nome do destinatário, o endereço, o valor e a moeda.
4. **Escolha o método de pagamento**: Escolha o método de pagamento que você deseja usar (por exemplo, cartão de crédito, depósito bancário, etc.).
5. **Confirme a remessa**: Verifique os detalhes da remessa e confirme a remessa.
6. **Pague a reme

In [ ]:
messages = [
    {"from": "human", "value": "Como criar uma conta no Brastel Remit?"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512, use_cache = True)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Como criar uma conta no Brastel Remit?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Para criar uma conta no Brastel Remit, você pode seguir os passos abaixo:

1. **Visite o site do Brastel Remit**: Acesse o site oficial do Brastel Remit em seu navegador de internet.
2. **Clique em "Cadastre-se"**: Localize o botão "Cadastre-se" ou "Registrar" na página inicial e clique nele.
3. **Preencha o formulário de cadastro**: Você será redirecionado para uma página com um formulário de cadastro. Preencha os campos com as informações solicitadas, como:
 * Nome completo
 * E-mail
 * Senha (e confirme-a)
 * CPF ou CNPJ (se for uma empresa)
 * Outras informações solicitadas (como telefone ou endereço)
4. **Leia e aceite os termos de uso**: Leia os termos de uso e as condições de serviço do Brastel Remit e aceite-o

## Carregando LLama 3 8B Quantizado (sem ser Instruct) para Fine Tuning

Quantization é uma técnica que reduz a precisão numérica dos parâmetros de um modelo de machine learning, geralmente convertendo valores de ponto flutuante de alta precisão (como 32 ou 16 bits) para representações de menor precisão (como 8 ou 4 bits). Isso diminui o tamanho do modelo e acelera sua execução, especialmente em hardware com memória limitada, como GPUs. Embora a quantization possa resultar em uma leve perda de precisão, ela geralmente mantém o desempenho geral do modelo enquanto melhora a eficiência computacional e reduz o uso de memória.

In [ ]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 96469696.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256, use_cache = True)

<|im_start|>user
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291171, 86267571212, 139583862445, 225851433717, 365435


## Carregando dados da Brastel

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/modulo11/fine-tuning/dataset.csv", encoding="utf-8", delimiter=";")
df = df[["Pergunta", "Resposta\n"]].dropna()
df = df.rename(columns={"Pergunta": "instruction", "Resposta\n": "output"})
df.head()

,instruction,output
0,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
3,Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
4,Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


## Preparando os dados para o formato exigido no Fine Tuning

In [ ]:
def convert_to_conversation_format(df):
    conversations = []
    for index, row in df.iterrows():
        conversation = [
            {'from': 'human', 'value': row['instruction']},
            {'from': 'gpt', 'value': row['output']}
        ]
        conversations.append(conversation)
    return {'conversations': conversations}

tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
jsonl = convert_to_conversation_format(df)
df_formatted = pd.DataFrame(jsonl)
dataset = Dataset.from_pandas(df_formatted)

NameError: name 'df' is not defined

In [ ]:
dataset = dataset.map(apply_template, batched=True)

In [ ]:
dataset

In [ ]:
dataset[:5]

## Fine Tuning com o LoRA (Low Rank Adapters) e SFT


LoRA (Low-Rank Adaptation) é uma técnica que permite o ajuste eficiente de grandes modelos de linguagem treinando apenas uma fração de seus parâmetros. Em vez de ajustar todos os bilhões de parâmetros de um modelo, LoRA treina pequenas matrizes de baixo rank, o que reduz significativamente o custo computacional e de memória. Essa abordagem mantém o modelo original praticamente inalterado, tornando o processo de fine-tuning muito mais rápido e barato.

A técnica QLoRA vai além ao quantizar o modelo para 4 bits, reduzindo ainda mais o tamanho e aumentando a eficiência sem comprometer drasticamente o desempenho. No exemplo, o modelo original de 16 GB foi reduzido para 5,4 GB. Isso permite o uso de hardware com menos capacidade de VRAM, mantendo a precisão através de formatos como NF4, suportados pela biblioteca bitsandbytes.

Além disso, rsLoRA (Rank-Stabilized LoRA) estabiliza o aprendizado em ranks mais altos, melhorando o desempenho do fine-tuning. Técnicas como checkpointing de gradiente ajudam a economizar ainda mais VRAM durante o processo. Com esses métodos, é possível ajustar grandes modelos, como os da família LLaMA, de forma eficiente e com menor custo de recursos, mantendo boa performance em tarefas específicas.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)


Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs=30,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="/content/drive/MyDrive/modulo11/fine-tuning/",
        seed=0,
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 24 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 90
 "-____-"     Number of trainable parameters = 41,943,040


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.91 GiB. GPU 

## Salvando modelo tunado no Hugging Face

In [ ]:
hf_token = userdata.get('HF_ACCESS_TOKEN')

In [ ]:
model.save_pretrained_merged("/content/drive/MyDrive/modulo11/fine-tuning/model", tokenizer, save_method="merged_16bit")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 52.12 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 65.28it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


In [ ]:
model.push_to_hub_merged("allancasadointeli/BrastelLlama-3.1-8B", tokenizer, save_method="merged_16bit", token=hf_token)

## Treinamento do modelo com Fine-Tunning usando GPU (5 iterações)

Nesta seção, será incorporado o treinamento do modelo fazendo uma comparativa usando os dois tipos de GPU, no caso, o tipo A100 e o T4.

Dentre os componenetes que serão avaliados será:

- Tempo de treinamento;
- Perda;
- Tempo por época;

Outrora, é importante citar instruções sobre como se faz o treinamento com ambas as tecnologias, com os passos abaixo:

1. Acesse a aba acima "Ambiente de execução";

2. Clique em alterar o tipo de ambiente de execução;

3. Selecione o tipo de GPU que deseja, mas como foi dito, o treinamento consistia na comparação do A100 e do T4;

### Callback para extrair os componentes necessários

In [ ]:
class CustomTrainerCallback(TrainerCallback):
    def __init__(self):
        self.start_time = None
        self.epoch_time = None
        self.total_time = 0
        self.epoch_times = []
        self.losses = []
        self.max_memory_used = 0  # Armazena o uso máximo de memória

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()
        print("Treinamento iniciado")

    def on_train_end(self, args, state, control, **kwargs):
        total_time = time.time() - self.start_time
        avg_epoch_time = sum(self.epoch_times) / len(self.epoch_times) if self.epoch_times else 0
        avg_loss = sum(self.losses) / len(self.losses) if self.losses else 0

        print(f"Treinamento finalizado. Tempo total: {total_time / 60:.2f} minutos")
        print(f"Tempo médio por época: {avg_epoch_time:.2f} segundos")
        print(f"Perda média (loss): {avg_loss:.4f}")
        print(f"Uso total de memória GPU: {self.max_memory_used:.2f} GB")

    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_time = time.time()

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch_duration = time.time() - self.epoch_time
        self.epoch_times.append(epoch_duration)
        print(f"Época {state.epoch} finalizada. Tempo decorrido: {epoch_duration:.2f} segundos")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            self.losses.append(logs['loss'])
            print(f"Perda (loss): {logs['loss']:.4f}")

        if torch.cuda.is_available():
            memory_used = torch.cuda.memory_allocated() / (1024 ** 3)  # Converte para GB
            self.max_memory_used = max(self.max_memory_used, memory_used)  # Atualiza o valor máximo
            print(f"Uso de memória GPU atual: {memory_used:.2f} GB")

### Treinamento do modelo com o callback

In [ ]:
custom_callback = CustomTrainerCallback()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs=30,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="/content/drive/MyDrive/modulo11/fine-tuning/",
        seed=0,
    ),
    callbacks=[custom_callback]
)

trainer.train()

### Análise comparativa da GPU A100 e da T4

Primeiramente, a média dos resultados obtidos:

No caso, da GPU A100:

| Treinamento Finalizado | Tempo Total (minutos) | Tempo Médio por Época (segundos) | Perda Média (loss) |
|-------------------------|------------------------|-----------------------------------|---------------------|
| 1                       | 4.70                   | 14.09                             | 0.0265              |
| 2                       | 4.50                   | 13.44                             | 0.0245              |
| 3                       | 4.62                   | 14.01                             | 0.0275              |
| 4                       | 4.28                   | 13.20                             | 0.0295              |
| 5                       | 4.48                   | 13.40                             | 0.0240              |

Na GPU T4:

Devido ao modelo exigir além de 15 GB de ram de memória durante a sua preparação e treinamento, ficou inviável treinar nesta configuração.

****

### Conclusão

A GPU A100 demonstrou um desempenho estável e eficiente no treinamento, com tempos consistentes e uma perda média razoável. Há espaço para otimizações na perda, o que pode ser explorado em futuros treinamentos.